# MUSA 5090 Final Project
This file is used for data exploration, analysis, feature engineering, and model buildings.


## Datasets

In [7]:
from google.colab import drive
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# read in properties dataset
opa_dir = '/content/drive/MyDrive/musa-final-proj/opa_properties_public.csv'
properties = pd.read_csv(opa_dir)
properties.head(200)

<ipython-input-4-0149586cf5df>:3: DtypeWarning: Columns (4,16,25,27,48,76) have mixed types. Specify dtype option on import or set low_memory=False.
  properties = pd.read_csv(opa_dir)


,objectid,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,...,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,shape
0,445763190,2022-05-24 12:13:13-04:00,NaN,NEC DOVER ST,NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,172.0,...,NaN,I,NaN,NaN,19132.0,CMX1,1001321879,NaN,NaN,SRID=2272;POINT ( 2689039.51193106 252319.375...
1,445763191,2022-05-24 12:13:13-04:00,NaN,SEC 27TH ST,NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,32.0,...,NaN,I,NaN,NaN,19146.0,RSA5,1001179696,NaN,NaN,SRID=2272;POINT ( 2686489.39071898 229330.622...
2,445763192,2022-05-24 12:13:13-04:00,NaN,NWC 16TH ST,NaN,T38,ROW B/OFF-STR 2 STY STONE,3,MIXED USE,373.0,...,NaN,I,1950.0,NaN,19145.0,CMX1,1001408261,820,ROW MIXED-COM/RES-BLT AS RES,SRID=2272;POINT ( 2690626.09996523 221499.086...
3,445763193,2022-05-24 12:13:13-04:00,NaN,"343'7 1/8"" W 31ST ST",NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,169.0,...,NaN,I,NaN,NaN,19132.0,RSA5,1001168670,NaN,NaN,SRID=2272;POINT ( 2687281.27004074 250386.258...
4,445763194,2022-05-24 12:13:13-04:00,NaN,197' W OF 19TH ST,NaN,ZL0,MISC FUNERAL HOME MASONRY,4,COMMERCIAL,134.0,...,NaN,A,1920.0,NaN,19130.0,RM1,1001494378,246,FUNERAL HOME,SRID=2272;POINT ( 2691553.59778215 240243.694...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,445763289,2022-05-24 12:13:13-04:00,NaN,"334'11 1/4 "" N",NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,164.0,...,NaN,I,NaN,NaN,19133.0,CMX2,1001598005,NaN,NaN,SRID=2272;POINT ( 2698753.98725273 249011.820...
196,445763290,2022-05-24 12:13:13-04:00,NaN,290' W OF 31ST ST,NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,151.0,...,NaN,I,NaN,NaN,19121.0,RSA5,1001408622,NaN,NaN,SRID=2272;POINT ( 2687105.63915840 248735.226...
197,445763291,2022-05-24 12:13:13-04:00,NaN,"74'7 3/4""N TIOGA",NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,201.0,...,NaN,I,NaN,NaN,19140.0,CMX2,1001625620,NaN,NaN,SRID=2272;POINT ( 2694809.80616473 256111.430...
198,445763292,2022-05-24 12:13:13-04:00,NaN,SW COR 50TH ST,NaN,S30,ROW W/OFF STR 2 STY MASON,3,MIXED USE,104.0,...,NaN,I,1925.0,NaN,19139.0,RSA5,1001412740,820,ROW MIXED-COM/RES-BLT AS RES,SRID=2272;POINT ( 2677389.92105706 240959.320...


In [3]:
# read in properties assessment history
ass_dir = '/content/drive/MyDrive/musa-final-proj/assessments.csv'
assessments = pd.read_csv(ass_dir)
assessments.head(200)

,parcel_number,year,market_value,taxable_land,taxable_building,exempt_land,exempt_building,objectid
0,11000001,2024,95000.0,95000.0,0.0,0.0,0.0,1372567274
1,11000001,2023,95000.0,95000.0,0.0,0.0,0.0,1372567275
2,11000002,2024,88000.0,88000.0,0.0,0.0,0.0,1372567276
3,11000002,2023,88000.0,88000.0,0.0,0.0,0.0,1372567277
4,11000003,2024,88000.0,88000.0,0.0,0.0,0.0,1372567278
...,...,...,...,...,...,...,...,...
195,11003700,2017,173500.0,45457.0,14543.0,0.0,113500.0,1372567469
196,11003700,2016,173500.0,7885.0,52115.0,0.0,113500.0,1372567470
197,11003700,2015,173500.0,7885.0,52115.0,0.0,113500.0,1372567471
198,11003805,2024,610100.0,122020.0,224517.0,0.0,263563.0,1372567472


In [9]:
# read in pwd parcels
parcels_dir = '/content/drive/MyDrive/musa-final-proj/phl_pwd_parcels.geojson'
parcels = gpd.read_file(parcels_dir)
parcels.head(200)

,OBJECTID,PARCELID,TENCODE,ADDRESS,OWNER1,OWNER2,BLDG_CODE,BLDG_DESC,BRT_ID,NUM_BRT,NUM_ACCOUNTS,GROSS_AREA,PIN,Shape__Area,Shape__Length,geometry
0,1,1,4900003710,3710 LANKENAU RD,RICKS LORRAINE,None,R30,ROW B/GAR 2STY MASONRY,521423200,1.0,1,1807,1.001311e+09,286.511719,97.246667,"POLYGON ((-75.20418 40.00218, -75.20419 40.002..."
1,2,2,7288004771,4771 SILVERWOOD ST,PAIMAN PAIMAN M,None,O30,ROW 2 STY MASONRY,211542000,1.0,1,1142,1.001486e+09,181.218750,64.261344,"POLYGON ((-75.22643 40.03475, -75.22663 40.034..."
2,3,3,5752003504,3504 MOUNT VERNON ST,COBB COLIN TIMOTHY,COBB MICHELLE ANN,O30,ROW 2 STY MASONRY,242059410,1.0,1,891,1.001382e+09,141.101562,68.554189,"POLYGON ((-75.19359 39.96420, -75.19363 39.964..."
3,4,4,8801006422,6422-24 N 11TH ST,BROWN E RICARDO,None,C40,DET W/D.GAR 2.5 STY MASON,493082500,1.0,1,4874,1.001613e+09,773.714844,129.816587,"POLYGON ((-75.13738 40.05034, -75.13782 40.050..."
4,5,5,8251003772,3772 WESTHAMPTON DR,WASHBURN ROBERT M,WASHBURN MARIE S,R31,ROW B/GAR 2STY MAS.+OTHER,663369700,1.0,1,2036,1.001561e+09,323.648438,94.870485,"POLYGON ((-74.96467 40.09322, -74.96497 40.093..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,228,2216002236,2236 N CHADWICK ST,MATTHEWS DELEPHINE,MATTHEWS DELPHINE,SR,VAC LAND RES < ACRE,161134800,1.0,1,696,1.001131e+09,110.437500,50.616118,"POLYGON ((-75.16038 39.98762, -75.16039 39.987..."
196,197,229,5488005839,5839 MCMAHON AVE,BOONE TAWANA L,None,H30,SEMI DET 2 STY MASONRY,591199000,1.0,1,2422,1.001362e+09,384.484375,99.304168,"POLYGON ((-75.16815 40.04598, -75.16838 40.045..."
197,198,230,2002010123,10123 BUSTLETON AVE,NAIR RADHAKRISHNAN R,NAIR SUMANGALA,C30,DET W/DET GAR 2 STY MASON,582533300,1.0,1,6803,1.001112e+09,1081.664062,142.900982,"POLYGON ((-75.02760 40.10380, -75.02760 40.103..."
198,199,231,4980005249,5249 LEBANON AVE,ENGLISH MURVIN,None,R30,ROW B/GAR 2STY MASONRY,521141100,1.0,1,2210,1.001319e+09,350.210938,109.616635,"POLYGON ((-75.22982 39.98839, -75.22981 39.988..."


Upon checking the dataset, we found the following attributes from the opa properties  might be useful to predicting property values.
- basements
- category code
- census tract
- central air
- exterior condition
- fireplaces
- garage spacess
- garage type
- interior condition
- market value
- number of bedrooms
- number of bathrooms
- number of rooms
- number stories
- parcel number
- quality grade
- sale price
- total area
- total livable area
- view
- year built
- shape (geometry)


In [14]:
# some baseline data processing work
properties_mdl = properties[['basements', #
                          'category_code',
                          'census_tract',
                          'central_air', #
                          'fireplaces', #
                          'garage_type', #
                          'market_value',
                          'number_of_bedrooms', #
                          'number_of_bathrooms', #
                          'number_of_rooms', #
                          'number_stories',#
                          'parcel_number',
                          'quality_grade', #
                          'sale_price',
                          'type_heater', #
                          'total_area', #
                          'total_livable_area', #
                          'view_type', #
                          'building_code_description_new', #
                          'year_built']] #

In [17]:
properties_mdl['Age'] = 2024 - properties_mdl['year_built']

properties_mdl['numRooms'] = np.select(
    [(properties_mdl['number_of_bedrooms'].isna()) & (~properties_mdl['number_of_bathrooms'].isna()),
     (properties_mdl['number_of_bathrooms'].isna()) & (~properties_mdl['number_of_bedrooms'].isna()),
      (properties_mdl['number_of_bathrooms'].isna()) & (properties_mdl['number_of_bedrooms'].isna())],
    [properties_mdl['number_of_bathrooms'],
     properties_mdl['number_of_bedrooms'],
     0],
    default=properties_mdl['number_of_bedrooms'] + properties_mdl['number_of_bathrooms']
)

properties_mdl['hasAC'] = np.where(properties_mdl['central_air'].isin(['1', 'Y']), 'Y', 'N')
properties_mdl['hasBasement'] = np.where(properties_mdl['basements'].isin(['1', '4', 'A', 'B', 'C', 'D', 'E', 'F']), 'Y', 'N')
properties_mdl['hasFireplace'] = np.where((properties_mdl['fireplaces'] == 0) | (properties_mdl['fireplaces'].isna()), 'N', 'Y')
properties_mdl['hasGarage'] = np.where((properties_mdl['garage_type'] == 0) | (properties_mdl['garage_type'].isna()), 'N', 'Y')
properties_mdl['stories'] = np.where(properties_mdl['number_stories'] == 1, 'single', np.where(properties_mdl['number_stories'] == 2, 'double', 'multiple'))
properties_mdl['area'] = np.where(properties_mdl['total_livable_area'] > properties_mdl['total_area'], properties_mdl['total_livable_area'], properties_mdl['total_area'])
properties_mdl['hasHeater'] = np.where((properties_mdl['type_heater'] == 0) | (properties_mdl['type_heater'].isna()), 'N', 'Y')
properties_mdl['quality'] = np.where(properties_mdl['quality_grade'].isin(['4', '5', '6', 'A', 'A+', 'A-', 'B', 'B+', 'B-', 'S', 'S+', 'X-']), 'Good', 'Bad')


properties_mdl['view'] = np.select(
    [properties_mdl['view_type'].isin(['I', '0']) | properties_mdl['view_type'].isna(),
     properties_mdl['view_type'].isin(['A', 'B', 'C'])],
    ['Typical','Scenic'],
    default='Urban'
)

properties_mdl['logarea'] = np.log(properties_mdl['area'])

<ipython-input-17-c59550975f01>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_mdl['Age'] = 2024 - properties_mdl['year_built']
<ipython-input-17-c59550975f01>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_mdl['numRooms'] = np.select(
<ipython-input-17-c59550975f01>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [21]:
# Convert the pandas Series of boolean values to numpy ndarray using .values
condition1 = properties_mdl['building_code_description_new'].str.contains('ROW', case=False).fillna(False).values
condition2 = properties_mdl['building_code_description_new'].str.contains('TWIN', case=False).fillna(False).values

properties_mdl['buildingdis'] = np.select(
    [
        condition1,
        condition2
    ],
    [
        'Row',
        'TWIN'
    ],
    default='Other'
)


<ipython-input-21-81fd3ebc4411>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_mdl['buildingdis'] = np.select(


In [23]:
properties_mdl

,basements,category_code,census_tract,central_air,fireplaces,garage_type,market_value,number_of_bedrooms,number_of_bathrooms,number_of_rooms,...,hasBasement,hasFireplace,hasGarage,stories,area,hasHeater,quality,view,logarea,buildingdis
0,NaN,6,172.0,NaN,NaN,NaN,15500.0,NaN,NaN,NaN,...,N,N,N,multiple,1194.0,N,Bad,Typical,7.085064,Other
1,NaN,6,32.0,NaN,NaN,NaN,120900.0,NaN,NaN,NaN,...,N,N,N,multiple,992.0,N,Bad,Typical,6.899723,Other
2,NaN,3,373.0,NaN,NaN,NaN,439000.0,NaN,NaN,NaN,...,N,N,N,multiple,2247.0,N,Bad,Typical,7.717351,Row
3,NaN,6,169.0,NaN,NaN,NaN,50000.0,NaN,NaN,NaN,...,N,N,N,multiple,800.0,N,Bad,Typical,6.684612,Other
4,NaN,4,134.0,NaN,NaN,NaN,1150500.0,NaN,NaN,NaN,...,N,N,N,multiple,4425.0,N,Bad,Scenic,8.395026,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582952,D,1,192.0,N,0.0,NaN,66200.0,3.0,1.0,NaN,...,Y,N,N,double,1176.0,Y,Bad,Typical,7.069874,Row
582953,H,1,390.0,N,1.0,NaN,116600.0,3.0,1.0,NaN,...,N,Y,N,double,980.0,Y,Bad,Typical,6.887553,Row
582954,NaN,1,143.0,NaN,NaN,NaN,409900.0,NaN,NaN,NaN,...,N,N,N,multiple,NaN,N,Bad,Typical,NaN,Other
582955,NaN,1,143.0,NaN,NaN,NaN,355000.0,NaN,NaN,NaN,...,N,N,N,multiple,NaN,N,Bad,Typical,NaN,Other


In [ ]:
# check attribtue name
properties.columns.tolist()

['objectid',
 'assessment_date',
 'basements',
 'beginning_point',
 'book_and_page',
 'building_code',
 'building_code_description',
 'category_code',
 'category_code_description',
 'census_tract',
 'central_air',
 'cross_reference',
 'date_exterior_condition',
 'depth',
 'exempt_building',
 'exempt_land',
 'exterior_condition',
 'fireplaces',
 'frontage',
 'fuel',
 'garage_spaces',
 'garage_type',
 'general_construction',
 'geographic_ward',
 'homestead_exemption',
 'house_extension',
 'house_number',
 'interior_condition',
 'location',
 'mailing_address_1',
 'mailing_address_2',
 'mailing_care_of',
 'mailing_city_state',
 'mailing_street',
 'mailing_zip',
 'market_value',
 'market_value_date',
 'number_of_bathrooms',
 'number_of_bedrooms',
 'number_of_rooms',
 'number_stories',
 'off_street_open',
 'other_building',
 'owner_1',
 'owner_2',
 'parcel_number',
 'parcel_shape',
 'quality_grade',
 'recording_date',
 'registry_number',
 'sale_date',
 'sale_price',
 'separate_utilities',
 '

In [5]:
# check attribtue name
assessments.columns.tolist()

['parcel_number',
 'year',
 'market_value',
 'taxable_land',
 'taxable_building',
 'exempt_land',
 'exempt_building',
 'objectid']

In [10]:
# check attribtue name
parcels.columns.tolist()

['OBJECTID',
 'PARCELID',
 'TENCODE',
 'ADDRESS',
 'OWNER1',
 'OWNER2',
 'BLDG_CODE',
 'BLDG_DESC',
 'BRT_ID',
 'NUM_BRT',
 'NUM_ACCOUNTS',
 'GROSS_AREA',
 'PIN',
 'Shape__Area',
 'Shape__Length',
 'geometry']

In [ ]:
# Read in census data
import requests

# Your API key
api_key = "77ba26a94bf747d61761544063606374da01af62"

# Base URL for Census API
url = "https://api.census.gov/data/2019/acs/acs5"

# Parameters for the API request
# Adjusting to fetch data for all census tracts within Philadelphia County (county code 101) in Pennsylvania (state code 42)
params = {
    "get": "B01003_001E",  # Total population
    'for': 'tract:*',  # All tracts
    'in': 'state:42 county:101',  # Within Philadelphia County, Pennsylvania
    "key": api_key
}

# Make the API request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()
    # Create a pandas DataFrame
    # The DataFrame includes the total population for each tract in Philadelphia County
    census = pd.DataFrame(data[1:], columns=data[0])
    # Optionally, convert tract and county codes to numeric and sort
    census[['state', 'county', 'tract']] = df[['state', 'county', 'tract']].apply(pd.to_numeric)
    census.sort_values(by=['tract'], inplace=True)
    print("Successful Download")
else:
    print(f"Failed to fetch data: {response.status_code}")

Successful Download


In [ ]:
census

,B01003_001E,state,county,tract
39,4118,42,101,100
241,2860,42,101,200
371,3385,42,101,300
276,2516,42,101,401
277,4097,42,101,402
...,...,...,...,...
159,0,42,101,980600
160,0,42,101,980700
161,0,42,101,980800
223,0,42,101,980900
